In [1]:
# !pip install pandas
# !pip install pinecone
# !pip unstall openai
# !pip install python-dotenv

In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

/Users/mpidev/miniconda3/envs/coffee-shop-env/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [40]:
MODEL_NAME= os.getenv("MODEL_NAME") 
MODEL_API_KEY: str = os.getenv("MODEL_API_KEY")
MODEL_BASE_URL: str = os.getenv("MODEL_BASE_URL")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") 
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME") 

In [20]:
pc = Pinecone(api_key=PINECONE_API_KEY)

client = OpenAI( 
    api_key=MODEL_API_KEY, 
    base_url=MODEL_BASE_URL
)

## Try out embeddings

In [24]:
output = client.embeddings.create(input = ["helloo there"],model=MODEL_NAME)
embeddings = output.data[0].embedding
print(embeddings)

[-0.005672971718013287, -0.020372934639453888, -0.01950583979487419, 0.004645442124456167, -0.019781799986958504, 0.018789241090416908, -0.0008100243285298347, -0.018215114250779152, 0.004085737746208906, 0.007962831296026707, 0.008273359388113022, 0.02585347555577755, 0.0031724662985652685, 0.011480250395834446, 0.007259970996528864, -0.002679419470950961, -5.3718806157121435e-05, 0.015439665876328945, -0.0021452484652400017, -0.0006106154178269207, -0.004909992218017578, -0.026708509773015976, 0.004209880251437426, 0.008335298858582973, 0.00015557929873466492, 0.0013396699214354157, 0.0076215555891394615, 0.017407149076461792, 0.014673792757093906, 0.0015102814650163054, 0.00880496297031641, -0.015313808806240559, -0.012726744636893272, -0.001465357025153935, -0.005414589773863554, -0.0076918695122003555, -0.005441516172140837, -0.0014472721377387643, -0.008048494346439838, 0.02803281880915165, 0.034365713596343994, -0.004803275689482689, 0.006964155472815037, -0.01081114076077938, -

In [25]:
len(embeddings)

960

## Wrangle dataset

In [26]:
df=pd.read_json('products/products.jsonl',lines=True)

In [27]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino_01.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,savory_scone_01.jpg


In [28]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [29]:
df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [30]:
texts = df['text'].tolist()

In [31]:
with open('products/milos_about_us.txt') as f:
    milos_about_section = f.read()
    
milos_about_section = "Coffee shop Milos about section: " + milos_about_section
texts.append(milos_about_section)

In [32]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

## Generate Embeddings

In [33]:
output = client.embeddings.create(input = texts,model=MODEL_NAME)

In [34]:
embeddings = output.data

## Push data to database

In [43]:
pc.create_index(
    name=PINECONE_INDEX_NAME,
    dimension=960, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [44]:
# Wait for the index to be ready
while not pc.describe_index(PINECONE_INDEX_NAME).status['ready']:
    time.sleep(1)

index = pc.Index(PINECONE_INDEX_NAME)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 20}

## Get Closest documents

In [45]:
output = client.embeddings.create(input = ["Is Cappuccino lactose-free?"],model=MODEL_NAME)
embeding = output.data[0].embedding

In [46]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Menu Items',
              'metadata': {'text': 'Menu Items: Menu Items\n'
                                   '\n'
                                   'Cappuccino - $4.50\n'
                                   'Jumbo Savory Scone - $3.25\n'
                                   'Latte - $4.75\n'
                                   'Chocolate Chip Biscotti - $2.50\n'
                                   'Espresso shot - $2.00\n'
                                   'Hazelnut Biscotti - $2.75\n'
                                   'Chocolate Croissant - $3.75\n'
                                   'Dark chocolate (Drinking Chocolate) - '
                                   '$5.00\n'
                                   'Cranberry Scone - $3.50\n'
                                   'Croissant - $3.25\n'
                                   'Almond Croissant - $4.00\n'
                                   'Ginger Biscotti - $2.50\n'
                                   'Oatmeal Scone - $3.